# Setup

In [1]:
"""
Page setup and execution for the aerospace mechanism chatbot
Example :        
-What can you tell me about latch mechanism design failures which have occurred        
-Follow up: Which one of the sources discussed volatile spherical joint interfaces           
"""
# import databutton as db

import os
import glob
import data_import
import queries
from dotenv import load_dotenv, find_dotenv
import streamlit as st
import openai
import secrets

from IPython.display import display, Markdown
import pprint

from dotenv import load_dotenv,find_dotenv

load_dotenv(find_dotenv(),override=True)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


True

In [2]:
# Check api keys
# print(os.getenv('OPENAI_API_KEY'))
# print(os.getenv('PINECONE_ENVIRONMENT'))
# print(os.getenv('PINECONE_API_KEY'))
# print(os.getenv('HUGGING_FACE_API_KEY'))
# print(os.getenv('LANGCHAIN_TRACING_V2'))
# print(os.getenv('LANGCHAIN_ENDPOINT'))
# print(os.getenv('LANGCHAIN_API_KEY'))
# print(os.getenv('LANGCHAIN_PROJECT'))

## Tradeable fields

In [3]:
output_level=None   # Max token limit, see section below
k=4 # Number of queries to return
search_type='similarity'    #  'mmr' or 'similarity'
temperature=0
verbose=True
chain_type='stuff'  # 'stuff' or  'map_reduce'

In [4]:
# Process some items
if output_level=="Concise":
    max_generated_tokens=50
elif output_level=="Detailed":
    max_generated_tokens=516
else:
    # max_generated_tokens = -1
    max_generated_tokens=None

# Track filtering, set history to be blank
message_id=0
filter_toggle=False # Filter sources on last answer
chat_history = []

## Vector database

### Pinecone

In [5]:
import pinecone

pinecone.init(
    api_key=os.getenv('PINECONE_API_KEY'),
    environment=os.getenv('PINECONE_ENVIRONMENT')
)
index_name = 'canopy--ams'

## Embeddings

### Openai Embeddings

In [6]:
from langchain.embeddings import OpenAIEmbeddings

embeddings_model = OpenAIEmbeddings(model="text-embedding-ada-002")

### Voyage embeddings

## LLM

Run one of the cells below to get an llm that plugs into the prompts section.

### Proprietary

#### Openai
Models: https://platform.openai.com/docs/models/gpt-3-5, https://platform.openai.com/docs/models/gpt-4-and-gpt-4-turbo

In [7]:
from langchain.chat_models import ChatOpenAI

openai.api_key = os.getenv('OPENAI_API_KEY')

# model_name='gpt-3.5-turbo-instruct'
model_name='gpt-3.5-turbo-1106' # 16,385 tokens
# model_name='gpt-4-0613'

llm = ChatOpenAI(temperature=temperature,
             model_name=model_name,
             max_tokens=max_generated_tokens,
             tags=[model_name+'-'+str(temperature)+'-'+str(max_generated_tokens)])

####  Anthropic, Claude

### Open Source

#### Ollama (local, works on mac only)
To get to work, download ollama and install on your mac.

In [8]:
from langchain.llms import Ollama

model_name='llama2:latest'
llm = Ollama(model=model_name)

In [9]:
# llm("the first man on the moon was...")

#### Hugging face models (via API)
* Choose a model_name from the leaderboard here: https://huggingface.co/spaces/lmsys/chatbot-arena-leaderboard 
* When testing these, many of the models will time out. 

In [44]:
from langchain.llms import HuggingFaceHub
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

model_name='google/flan-t5-xxl'   # Fast but with uninteresting answers
# model_name='tiiuae/falcon-40b'  # Slow
# model_name='WizardLM/WizardLM-70B-V1.0'   # Slow
# model_name='HuggingFaceH4/zephyr-7b-beta' # Slow
# model_name='meta-llama/Llama-2-70b-chat-hf'

llm = HuggingFaceHub(repo_id=model_name,
                     model_kwargs={"temperature": 0.1, "max_length": 250}
)

question = "Who won the FIFA World Cup in the year 1994? "
template = """Question: {question}

Answer: Let's think step by step."""
prompt = PromptTemplate(template=template, input_variables=["question"])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [45]:
llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.run(question))

The 1994 FIFA World Cup was held in France. France won the 1994 FIFA World Cup. The answer: France.


In [41]:
text="""
Generate a question that a human would ask to get the following response from a chatbot who is knowledgable about aerospace mechanisms:
Dynamic Behavior of Ball Bearings under Axial Vibration  Virgil Hinque* and René Seiler* Abstract  The paper addresses the dynamics of ball bearings when exposed to vibration loads along their axis of  rotation. Following common practice in space mechanisms design, the bearings are mounted in either hard  preloaded or soft preloaded pairs. A computer-based model has been developed for the analysis and  prediction of the load-deflection characteristics in bearing systems. Furthermore, the model may be used  to quantify the maximum loads applied on the bearings and the resulting stresses during a vibration test or  a spacecraft launch.  In parallel to the model development, an experimental test program has been carried out in order to get  sufficient data for model correlation. In this context, the paper also elaborates on the post-processing of the  acquired test signals and discusses specific effects, for instance nonlinearities due to the use of snubbers,  in the time domain as well as in the frequency domain.  Introduction  Many space mechanisms use ball bearings for rotation functions. Therefore, assessing the bearing  performance for the relevant environmental conditions is one of the typical challenges faced during the  equipment design process. In this frame, it is common engineering practice to reduce the effect of a sine  and random vibration environment to quasi-static equivalent loads and stresses. The relevant ball bearing  systems often comprise two identical deep-groove or angular-contact bearings in an axially preloaded  configuration. Several studies on the influence of the preload and other parameters on the structural  behavior of such bearing assemblies have been done by the European Space Tribology Laboratory (ESTL).  In a recent investigation, 25 ball bearing cartridges (“test units” or “bearing housings”) with different preload  and snubber configurations were submitted to a series of sine and random vibration tests. The discussion  of findings was mainly based on the analysis of frequency-domain data and bearing damage assessment  via visual inspection [1].  The ESTL investigation inspired a number of ideas for continuation of the research, among others the  development of a computer-based model that would be able to simulate the behavior of the bearing  cartridges, especially those showing nonlinear features in their response. An adequate model should be  able to predict the load transmission across the bearings in static and dynamic load situations. As the main  sizing criterion for ball bearings is based on the allowable peak Hertzian contact pressure between the balls  and the races [2], accurate knowledge of the maximum bearing loads is a key aspect for successful bearing  selection and implementation in a space mechanism.  During the current investigation at the European Space Research and Technology Centre (ESTEC), a  model was built using MATLAB®/Simulink®, with only the axial degree of freedom in a bearing taken into  consideration. Because model correlation with real test results is of importance, a test program  complementary to that reported in [1] has been conducted, with specific focus on the acquisition and  interpretation of time-domain data. The following chapters describe the computer-based model, the design  of the test units, as well as the details of the test campaign and corresponding results. The last part of the  paper is dedicated to the comparison between the model output and the experimental test data.  *European Space Agency (ESA/ESTEC), Noordwijk, The Netherlands Proceedings of the 44th Aerospace Mechanisms Symposium, NASA Glenn Research Center, May 16-18, 2018 NASA/CP—2018-219887 83
"""
print(llm(text))

What is the abstract about?


# Prompts

In [ ]:
prompt=['Prompt 1: What types of lubricants are to be avoided when designing space mechanisms?',
        'Prompt 2: Can you speak to what failures have occurred when using mineral oil lubricants?']

## First prompt

In [ ]:
# Set up object
tags=['prompt1']
qa_model_obj=queries.QA_Model(index_name,
                    embeddings_model,
                    llm,
                    k,
                    search_type,
                    verbose,
                    filter_arg=filter_toggle)

# Generate a response using your chat model
qa_model_obj.query_docs(prompt[0],tags)
ai_response=qa_model_obj.result['answer']
references=qa_model_obj.sources[-1]

display(Markdown(prompt[0]))
display(Markdown(ai_response))
pprint.pprint(references,width=500)

## Second Prompt

In [ ]:
# Generate a response using your chat model
tags=['prompt2_nofilter']
qa_model_obj.update_model(llm=llm,
                          filter_arg=False)

qa_model_obj.query_docs(prompt[1],
                        tags=tags)
ai_response=qa_model_obj.result['answer']
references=qa_model_obj.sources[-1]

display(Markdown(prompt[1]))
display(Markdown(ai_response))
pprint.pprint(references,width=500)

# Langsmith

In [ ]:
# from langchain.chat_models import ChatOpenAI

# llm=ChatOpenAI()
# llm.invoke("Hello world!")